In [215]:
import pandas as pd
import numpy as np
wins = pd.read_csv('csv/sql/wins.csv')
team_games = pd.read_csv("csv/sql/team_games.csv", dtype= {'teamid': 'str'})

df = team_games.merge(wins, on="platformgameid")


df['win'] = np.where(df['side'] == df['winningteam'], 1, 0)
df.drop(columns=["winningteam"], inplace=True)
print(len(df))
display(df.head())

blue_wins = df[(df["win"] == 1) & (df["side"] == 100)]
red_wins = df[(df["win"] == 1) & (df["side"] == 200)]
print("Total Games: \t", len(df["esportsgameid"].unique()))
print("Blue wins: \t", len(blue_wins))
print("Red wins: \t", len(red_wins))
print("Blue win rate: \t", len(blue_wins)/len(wins))
print("Red win rate: \t", len(red_wins)/len(wins))

50462


,teamid,esportsgameid,platformgameid,side,win
0,105550033967461806,110378429158160204,ESPORTSTMNT02:3220280,200,1
1,105550059790656435,110378429158160204,ESPORTSTMNT02:3220280,100,0
2,110424397251619481,110424377526038803,ESPORTSTMNT06:2818303,200,0
3,105520796734358045,110424377526038803,ESPORTSTMNT06:2818303,100,1
4,99566404856367466,110428848767875084,LPL_B:326759,200,1


Total Games: 	 25223
Blue wins: 	 13336
Red wins: 	 11895
Blue win rate: 	 0.5280538507226292
Red win rate: 	 0.4709958424074441


In [216]:
first_blood = pd.read_csv("csv/sql/first_blood.csv")
first_blood = first_blood.sort_values(by=["platformgameid"])
first_blood = first_blood.drop_duplicates(subset=['platformgameid'], keep='first')
first_blood = first_blood.rename(columns={
    "gametime": "first_blood_time",
})
df = df.merge( first_blood, on="platformgameid" )
df['first_blood'] = np.where(df['side'] == df['killerteamid'], 1, 0)
df.drop(columns=["killerteamid"], inplace=True)

def wr_for_side(stat, side):
    side_name = "Blue" if side == 100 else "Red"
    blue_fb = df[(df[stat] == 1) & (df["side"] == side)]
    blue_fb_wins = df[(df[stat] == 1) & (df["side"] == side) & (df["win"] == 1)]
    print(f"{side_name} {stat}: \t", len(blue_fb))
    print(f"{side_name} {stat} wins: \t", len(blue_fb_wins))
    print(f"{side_name} {stat} win rate: \t", len(blue_fb_wins)/len(blue_fb))

wr_for_side(stat="first_blood", side=100)
wr_for_side(stat="first_blood", side=200)



Blue first_blood: 	 12661
Blue first_blood wins: 	 6739
Blue first_blood win rate: 	 0.5322644340889345
Red first_blood: 	 12499
Red first_blood wins: 	 5937
Red first_blood win rate: 	 0.4749979998399872


In [217]:
first_building = pd.read_csv("csv/sql/first_buildings.csv")
first_building  = first_building[["platformgameid", "gametime", "teamid", "buildingtype"]]

first_tower = first_building[first_building["buildingtype"] == "turret"].copy()
first_tower = first_tower.sort_values(by=["platformgameid", "gametime"])
first_tower = first_tower.drop_duplicates(subset=['platformgameid'], keep='first')
# killer_team_id = first_tower["teamid"].apply(lambda x: 200 if x == 100 else 100)
first_tower["first_tower_owner_id"] = first_tower["teamid"]
first_tower = first_tower.drop(columns=["teamid", "buildingtype"])
first_tower = first_tower.rename(columns={
    "gametime": "first_tower_time",
})
print("First tower: \t", len(first_tower))
display(first_tower.head())
first_inhibitor = first_building[first_building["buildingtype"] == "inhibitor"].copy()
first_inhibitor = first_inhibitor.sort_values(by=["platformgameid", "gametime"])
first_inhibitor = first_inhibitor.drop_duplicates(subset=['platformgameid'], keep='first')

first_inhibitor["first_inhibitor_owner_id"] = first_inhibitor["teamid"]
first_inhibitor = first_inhibitor.drop(columns=["teamid", "buildingtype"])
first_inhibitor = first_inhibitor.rename(columns={
    "gametime": "first_inhibitor_time",
})
print("First inhibitor: \t", len(first_inhibitor))
first_inhibitor.head()

First tower: 	 25252


,platformgameid,first_tower_time,first_tower_owner_id
40575,ESPORTSTMNT01:1110148,939441,200
23132,ESPORTSTMNT01:1110162,677086,200
31938,ESPORTSTMNT01:1110177,836608,200
12714,ESPORTSTMNT01:1110187,626524,100
39594,ESPORTSTMNT01:1110198,895424,200


First inhibitor: 	 25212


,platformgameid,first_inhibitor_time,first_inhibitor_owner_id
70824,ESPORTSTMNT01:1110148,1708748,200
41143,ESPORTSTMNT01:1110162,1602302,200
62898,ESPORTSTMNT01:1110177,1546286,200
3623,ESPORTSTMNT01:1110187,1628728,200
50722,ESPORTSTMNT01:1110198,1694081,200


In [218]:
df = df.merge(first_tower, on="platformgameid")
df = df.merge(first_inhibitor, on="platformgameid")
df['first_tower'] = np.where(df['side'] != df['first_tower_owner_id'], 1, 0)
df.drop(columns=["first_tower_owner_id"], inplace=True)
df['first_inhibitor'] = np.where(df['side'] != df['first_inhibitor_owner_id'], 1, 0)
df.drop(columns=["first_inhibitor_owner_id"], inplace=True)
df.head()

,teamid,esportsgameid,platformgameid,side,win,first_blood_time,first_blood,first_tower_time,first_inhibitor_time,first_tower,first_inhibitor
0,105550033967461806,110378429158160204,ESPORTSTMNT02:3220280,200,1,509356,0,945005,1860636,1,1
1,105550059790656435,110378429158160204,ESPORTSTMNT02:3220280,100,0,509356,1,945005,1860636,0,0
2,110424397251619481,110424377526038803,ESPORTSTMNT06:2818303,200,0,242551,1,778010,1406628,0,0
3,105520796734358045,110424377526038803,ESPORTSTMNT06:2818303,100,1,242551,0,778010,1406628,1,1
4,99566404856367466,110428848767875084,LPL_B:326759,200,1,259808,0,947875,1447813,1,1


In [219]:
wr_for_side(stat="first_tower", side=100)
wr_for_side(stat="first_tower", side=200)

Blue first_tower: 	 13837
Blue first_tower wins: 	 9632
Blue first_tower win rate: 	 0.6961046469610465
Red first_tower: 	 11353
Red first_tower wins: 	 7672
Red first_tower win rate: 	 0.6757685193340968


In [220]:
event_start_times = pd.read_csv("csv/sql/event_start_times.csv")
event_start_times["event_start"] = pd.to_datetime(event_start_times["eventtime"], errors="coerce")
df = df.merge(event_start_times, on="platformgameid")
df.head()

,teamid,esportsgameid,platformgameid,side,win,first_blood_time,first_blood,first_tower_time,first_inhibitor_time,first_tower,first_inhibitor,eventtime,event_start
0,105550033967461806,110378429158160204,ESPORTSTMNT02:3220280,200,1,509356,0,945005,1860636,1,1,2023-07-13T05:05:52.820Z,2023-07-13 05:05:52.820000+00:00
1,105550059790656435,110378429158160204,ESPORTSTMNT02:3220280,100,0,509356,1,945005,1860636,0,0,2023-07-13T05:05:52.820Z,2023-07-13 05:05:52.820000+00:00
2,110424397251619481,110424377526038803,ESPORTSTMNT06:2818303,200,0,242551,1,778010,1406628,0,0,2023-07-06T19:29:32.607Z,2023-07-06 19:29:32.607000+00:00
3,105520796734358045,110424377526038803,ESPORTSTMNT06:2818303,100,1,242551,0,778010,1406628,1,1,2023-07-06T19:29:32.607Z,2023-07-06 19:29:32.607000+00:00
4,99566404856367466,110428848767875084,LPL_B:326759,200,1,259808,0,947875,1447813,1,1,2023-07-03T09:15:36.554Z,2023-07-03 09:15:36.554000+00:00


In [221]:
WINDOW = 6

df = df.sort_values(by=['teamid', 'event_start'])
# Group the DataFrame by 'team_id'
grouped = df.groupby('teamid')
# Calculate the rolling average for each group (team) using a window of 5
df['win_avg'] = grouped['win'].rolling(window=WINDOW).mean().reset_index(0, drop=True)
df['first_blood_avg'] = grouped['first_blood'].rolling(window=WINDOW).mean().reset_index(0, drop=True)
df['first_tower_avg'] = grouped['first_tower'].rolling(window=WINDOW).mean().reset_index(0, drop=True)
df['first_inhibitor_avg'] = grouped['first_inhibitor'].rolling(window=WINDOW).mean().reset_index(0, drop=True)
df['first_blood_time_avg'] = grouped['first_blood_time'].rolling(window=WINDOW).mean().reset_index(0, drop=True)
df['first_tower_time_avg'] = grouped['first_tower_time'].rolling(window=WINDOW).mean().reset_index(0, drop=True)
df['first_inhibitor_time_avg'] = grouped['first_inhibitor_time'].rolling(window=WINDOW).mean().reset_index(0, drop=True)


g2 = df[df["teamid"] =="98767991926151025"].sort_values(by=["event_start"])
display(g2[["platformgameid","first_blood","side","eventtime", "first_blood_avg", "first_tower_avg", "first_inhibitor_avg"]].head(10))
# display(df)

,platformgameid,first_blood,side,eventtime,first_blood_avg,first_tower_avg,first_inhibitor_avg
35677,ESPORTSTMNT01:1120949,0,100,2019-05-10T10:13:36.715Z,NaN,NaN,NaN
19590,ESPORTSTMNT01:1121076,1,200,2019-05-10T15:14:36.530Z,NaN,NaN,NaN
49281,ESPORTSTMNT01:1121126,0,100,2019-05-11T07:11:40.727Z,NaN,NaN,NaN
25320,ESPORTSTMNT01:1121178,0,200,2019-05-11T11:06:57.969Z,NaN,NaN,NaN
49620,ESPORTSTMNT01:1121276,0,200,2019-05-12T09:14:37.131Z,NaN,NaN,NaN
16564,ESPORTSTMNT01:1121298,0,200,2019-05-12T11:07:24.519Z,0.166667,0.333333,0.666667
38278,ESPORTSTMNT01:1121364,1,100,2019-05-13T10:11:44.670Z,0.333333,0.166667,0.666667
21929,ESPORTSTMNT01:1121389,0,100,2019-05-13T12:11:30.074Z,0.166667,0.166667,0.833333
43018,ESPORTSTMNT01:1121458,1,100,2019-05-14T11:05:47.540Z,0.333333,0.166667,0.666667
12218,ESPORTSTMNT01:1121470,0,200,2019-05-14T12:55:18.853Z,0.333333,0.000000,0.500000


In [222]:
blue_side = df[df["side"] == 100]
blue_side = blue_side.rename(columns={
    "teamid": "blue_teamid",
})
blue_side.drop(columns=["side"], inplace=True)

red_side = df[df["side"] == 200]
red_side = red_side.rename(columns={
    "teamid": "red_teamid",
})
red_side.drop(columns=["side", "esportsgameid"], inplace=True)

df = blue_side.merge(red_side, on="platformgameid", suffixes=("_blue", "_red"))
df['winningteam'] = np.where(df['win_blue'] == 1, 100, 200)
df.drop(columns=["win_blue", "win_red"], inplace=True)
df.head()

,blue_teamid,esportsgameid,platformgameid,first_blood_time_blue,first_blood_blue,first_tower_time_blue,first_inhibitor_time_blue,first_tower_blue,first_inhibitor_blue,eventtime_blue,...,eventtime_red,event_start_red,win_avg_red,first_blood_avg_red,first_tower_avg_red,first_inhibitor_avg_red,first_blood_time_avg_red,first_tower_time_avg_red,first_inhibitor_time_avg_red,winningteam
0,100205573495116443,102181142519049888,ESPORTSTMNT01:1130786,1361826,1,1048277,1972924,0,0,2019-06-05T11:06:59.959Z,...,2019-06-05T11:06:59.959Z,2019-06-05 11:06:59.959000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200
1,100205573495116443,102181142519246497,ESPORTSTMNT01:1130806,564310,1,781709,1977250,1,0,2019-06-05T12:03:48.003Z,...,2019-06-05T12:03:48.003Z,2019-06-05 12:03:48.003000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200
2,100205573495116443,102181142519377570,ESPORTSTMNT01:1140917,546257,1,1166569,1915734,1,1,2019-06-05T12:59:28.946Z,...,2019-06-05T12:59:28.946Z,2019-06-05 12:59:28.946000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100
3,100205573495116443,102181142541397833,ESPORTSTMNT01:1143282,225339,1,769258,1844212,0,1,2019-06-13T08:17:52.821Z,...,2019-06-13T08:17:52.821Z,2019-06-13 08:17:52.821000+00:00,0.166667,0.166667,0.666667,0.333333,672339.000000,933979.500000,1922684.5,100
4,100205573495116443,102181142541463370,ESPORTSTMNT01:1143414,413398,1,893223,1445342,1,0,2019-06-13T09:10:57.233Z,...,2019-06-13T09:10:57.233Z,2019-06-13 09:10:57.233000+00:00,0.333333,0.000000,0.666667,0.500000,617193.166667,924737.833333,1883194.5,200


In [236]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np

df.fillna(0, inplace=True)
X = df[[
        "first_blood_avg_blue", "first_tower_avg_blue", "first_inhibitor_avg_blue",
        "first_blood_avg_red", "first_tower_avg_red", "first_inhibitor_avg_red",
        # "first_blood_time_avg_blue", "first_tower_time_avg_blue", "first_inhibitor_time_avg_blue",
        # "first_blood_time_avg_red", "first_tower_time_avg_red", "first_inhibitor_time_avg_red",
        # "blue_avg_shutdown_converted", "red_avg_shutdown_converted",
        # "blue_avg_shutdown_held", "red_avg_shutdown_held", "blue_avg_shutdown_collected", "red_avg_shutdown_collected",
]]
y = df["winningteam"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

gnb = GaussianNB()
model = gnb.fit(X_train,y_train)
predictive_labels = gnb.predict(X_test)

print(accuracy_score(y_test, predictive_labels))

0.7200396825396825
